## Import Libraries

In [18]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [19]:
import matplotlib.pyplot as plt
plt.ion()

In [20]:
from pydgilib_extra import *

## Data Logging

Create a figure for the plot.

In [4]:
fig = plt.figure(figsize=(10, 6))
fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [5]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_values": [True, True, True, True],
    "gpio_delay_time" : 0.0007,
    "plot_pins_method": "line",
    "plot_xmax": 15,
    "fig": fig,
    "window_title": "Experiment AES",
    "file_name_base": "experiment_aes"
}

Stop criteria to pass to the logger:

In [6]:
#MIN_LOG_DURATION = 0.5 # seconds
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1]) #and logger_data.gpio.timestamps[-1] > MIN_LOG_DURATION
# def stop_fn(logger_data):
#     return len(logger_data.gpio) > 30

Perform the measurement.

In [7]:
data = []
fig.clf()
for ax in fig.get_axes():
    ax.cla()

with DGILibExtra(**config_dict) as dgilib:
    dgilib.device_reset()
#     data = dgilib.logger.log(config_dict["plot_xmax"],stop_fn)
    dgilib.logger.log(100,stop_fn)
    data = dgilib.data

D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0
  ax.set_xlim((valmin, valmax))


## Analysis

In [8]:
encrypt_charge, encrypt_time = power_and_time_per_pulse(data, 2, 0.01)

In [9]:
decrypt_charge, decrypt_time = power_and_time_per_pulse(data, 3, 0.01)

In [10]:
MBEDTLS_AES_BLOCK_SIZE = 16
MIN_AES_BLOCKS = 1
num_bytes = range(MIN_AES_BLOCKS * MBEDTLS_AES_BLOCK_SIZE, MBEDTLS_AES_BLOCK_SIZE * (MIN_AES_BLOCKS + len(encrypt_charge)), MBEDTLS_AES_BLOCK_SIZE)
print(f"MAX_AES_BLOCKS: {MIN_AES_BLOCKS + len(encrypt_charge) - 1}")

MAX_AES_BLOCKS: 100


In [11]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [12]:
results = []
for y in [encrypt_charge, decrypt_charge, encrypt_time, decrypt_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 6.5784e-14
    reduced chi-square = 6.7127e-16
    Akaike info crit   = -3491.75645
    Bayesian info crit = -3486.54611
[[Variables]]
    slope:      3.2411e-08 +/- 5.6098e-12 (0.02%) (init = 0)
    intercept: -1.7168e-08 +/- 5.2209e-09 (30.41%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.868

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 4.9986e-14
    reduced chi-square = 5.1007e-16
    Akaike info crit   = -3519.21958
    Bayesian info crit = -3514.00924
[[Variables]]
    slope:      3.3401e-08 +/- 4.8900e-12 (0.01%) (init = 0)
    intercept: -1.5214e-08 +/- 4.5510e-09 (29.91%) (init = 1)
[[Correlations]] (unrep

In [13]:
fig2 = plt.figure(figsize=(9, 6))
fig2.canvas.set_window_title("Analysis AES")

In [14]:
charge_color = 'r'
time_color = 'b'

In [15]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Total Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [16]:
lines = []
lines += ax1.plot(num_bytes, encrypt_charge, charge_color+'-', label='Encrypt Charge')
lines += ax1.plot(num_bytes, decrypt_charge, charge_color+'--', label='Decrypt Charge')
lines += ax2.plot(num_bytes, encrypt_time, time_color+'-', label='Encrypt Time')
lines += ax2.plot(num_bytes, decrypt_time, time_color+'--', label='Decrypt Time')
ax1.legend(handles=lines)
ax1.set_title(f"Encrypt Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"Decrypt Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [17]:
# data